In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
#data
#! wget https://dl.fbaipublicfiles.com/GENRE/lang_title2wikidataID-normalized_with_redirect.pkl
    
with open("lang_title2wikidataID-normalized_with_redirect.pkl", "rb") as f:
    lang_title2wikidataID = pickle.load(f)

In [3]:
import numpy as np
import scipy.stats
import math
import torch
from torch import nn
import pandas as pd
import matplotlib.pyplot as plt
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import ConstrainedBeamSearchScorer
from transformers import LogitsProcessorList
from transformers import MinLengthLogitsProcessor

from tqdm import tqdm
import pickle
from wikidataintegrator import wdi_core
from wikidata.client import Client
import wikidata
from itertools import compress
from time import time

import en_core_web_sm
import spacy
import warnings
warnings.filterwarnings("ignore")

# spacy.prefer_gpu()
nlp = spacy.load("en_core_web_sm")

#nlp = en_core_web_sm.load()
client = wikidata.client.Client()

softmax = nn.Softmax()

In [5]:
# with open("lang_title2wikidataID-normalized_with_redirect.pkl", "rb") as f:
#     lang_title2wikidataID = pickle.load(f)
    
# data = list(lang_title2wikidataID)[:len(lang_title2wikidataID)]
# data = np.array(data)
# en_data = data[data[:,0] == "en"][:,1]
# len(en_data)


# tokenized_data = []

# for word in tqdm(en_data):
#     tokenized_data.append(tokenizer(word)['input_ids'])
    
# # saving english tokenized_data from lang_title2wikidataID
# # np_array_tokenized_data = np.array(tokenized_data)
# # np.save("eng_tokenized_lang_title2wikidataID_by_t5_small_ssm_nq.npy", np_array_tokenized_data)

In [6]:
model = AutoModelForSeq2SeqLM.from_pretrained("google/t5-small-ssm-nq")
tokenizer = AutoTokenizer.from_pretrained("google/t5-small-ssm-nq")

device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
model.to(device)


T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              (wo): 

In [7]:
tokenized_data = np.load("eng_tokenized_lang_title2wikidataID_by_t5_small_ssm_nq.npy", allow_pickle=True)

# constraint beam search via trie

In [ ]:
# !git clone https://github.com/huggingface/transformers.git
# %cd transformers
# !pip install -e .

In [9]:
# some_file.py
import sys
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, '/home/petrakov/Experiments_with_UE/transformers/src/transformers/generation')



In [10]:
import beam_constraints

In [11]:
start = time()
beam_constrained_trie = beam_constraints.DisjunctiveTrie(list(tokenized_data), no_subsets=False)
end = time()

print("Duration of trie construction:", np.round(end - start, 4), "sec.")

Start DisjunctiveTrie
Duration of trie construction: 72.0465 sec.


# generation using our trie

In [7]:
from abc import ABC, abstractmethod
from typing import List, Optional


class Constraint(ABC):
    r"""Abstract base class for all constraints that can be applied during generation.
    It must define how the constraint can be satisfied.

    All classes that inherit Constraint must follow the requirement that

    ```py
    completed = False
    while not completed:
        _, completed = constraint.update(constraint.advance())
    ```

    will always terminate (halt).
    """

    def __init__(self):
        # test for the above condition
        self.test()

    def test(self):
        """
        Tests whether this constraint has been properly defined.
        """
        counter = 0
        completed = False
        while not completed:
            if counter == 1:
                self.reset()
            advance = self.advance()
            if not self.does_advance(advance):
                raise Exception(
                    "Custom Constraint is not defined correctly. self.does_advance(self.advance()) must be true."
                )

            stepped, completed, reset = self.update(advance)
            counter += 1

            if counter > 10000:
                raise Exception("update() does not fulfill the constraint.")

        if self.remaining() != 0:
            raise Exception("Custom Constraint is not defined correctly.")

    @abstractmethod
    def advance(self):
        """
        When called, returns the token that would take this constraint one step closer to being fulfilled.

        Return:
            token_ids(`torch.tensor`): Must be a tensor of a list of indexable tokens, not some integer.
        """
        raise NotImplementedError(
            f"{self.__class__} is an abstract class. Only classes inheriting this class can be called."
        )

    @abstractmethod
    def does_advance(self, token_id: int):
        """
        Reads in a token and returns whether it creates progress.
        """
        raise NotImplementedError(
            f"{self.__class__} is an abstract class. Only classes inheriting this class can be called."
        )

    @abstractmethod
    def update(self, token_id: int):
        """
        Reads in a token and returns booleans that indicate the progress made by it. This function will update the
        state of this object unlikes `does_advance(self, token_id: int)`.

        This isn't to test whether a certain token will advance the progress; it's to update its state as if it has
        been generated. This becomes important if token_id != desired token (refer to else statement in
        PhrasalConstraint)

        Args:
            token_id(`int`):
                The id of a newly generated token in the beam search.
        Return:
            stepped(`bool`):
                Whether this constraint has become one step closer to being fulfuilled.
            completed(`bool`):
                Whether this constraint has been completely fulfilled by this token being generated.
            reset (`bool`):
                Whether this constraint has reset its progress by this token being generated.
        """
        raise NotImplementedError(
            f"{self.__class__} is an abstract class. Only classes inheriting this class can be called."
        )

    @abstractmethod
    def reset(self):
        """
        Resets the state of this constraint to its initialization. We would call this in cases where the fulfillment of
        a constraint is abrupted by an unwanted token.
        """
        raise NotImplementedError(
            f"{self.__class__} is an abstract class. Only classes inheriting this class can be called."
        )

    @abstractmethod
    def remaining(self):
        """
        Returns the number of remaining steps of `advance()` in order to complete this constraint.
        """
        raise NotImplementedError(
            f"{self.__class__} is an abstract class. Only classes inheriting this class can be called."
        )

    @abstractmethod
    def copy(self, stateful=False):
        """
        Creates a new instance of this constraint.

        Args:
            stateful(`bool`): Whether to not only copy the constraint for new instance, but also its state.

        Return:
            constraint(`Constraint`): The same constraint as the one being called from.
        """
        raise NotImplementedError(
            f"{self.__class__} is an abstract class. Only classes inheriting this class can be called."
        )

class DisjunctiveTrie:
    def __init__(self, nested_token_ids: List[List[int]], no_subsets=False):
        r"""
        A helper class that builds a trie with the words represented in `nested_token_ids`.
        """
        print("Start DisjunctiveTrie")
        self.max_height = max([len(one) for one in nested_token_ids])

        root = dict()
        for token_ids in nested_token_ids:
            level = root
            for tidx, token_id in enumerate(token_ids):
                if token_id not in level:
                    level[token_id] = dict()

                level = level[token_id]

        if no_subsets and self.has_subsets(root, nested_token_ids):
            raise ValueError(
                "Each list in `nested_token_ids` can't be a complete subset of another list, but is"
                f" {nested_token_ids}."
            )

        self.trie = root

    def next_tokens(self, current_seq):
        """
        The next possible tokens that will progress the trie, given the current sequence of tokens in `current_seq`.
        """
        start = self.trie

        for current_token in current_seq:
            start = start[current_token]

        next_tokens = list(start.keys())

        return next_tokens

    def reached_leaf(self, current_seq):
        next_tokens = self.next_tokens(current_seq)

        return len(next_tokens) == 0

    def count_leaves(self, root):
        next_nodes = list(root.values())
        if len(next_nodes) == 0:
            return 1
        else:
            return sum([self.count_leaves(nn) for nn in next_nodes])

    def has_subsets(self, trie, nested_token_ids):
        """
        Returns whether # of leaves == # of words. Otherwise some word is a subset of another.
        """
        leaf_count = self.count_leaves(trie)
        return len(nested_token_ids) != leaf_count


class DisjunctiveConstraint(Constraint):
    r"""
    A special [`Constraint`] that is fulfilled by fulfilling just one of several constraints.

    Args:
        nested_token_ids (`List[List[int]]`): a list of words, where each word is a list of ids. This constraint
        is fulfilled by generating just one from the list of words.
    """

    def __init__(self, nested_token_ids: List[List[int]]):
        super(Constraint, self).__init__()

        
        print("start DisjunctiveConstraint")
        if not isinstance(nested_token_ids, list) or len(nested_token_ids) == 0:
            raise ValueError(f"`nested_token_ids` has to be a non-empty list, but is {nested_token_ids}.")
        if any(not isinstance(token_ids, list) for token_ids in nested_token_ids):
            raise ValueError(f"`nested_token_ids` has to be a list of lists, but is {nested_token_ids}.")
        if any(
            any((not isinstance(token_id, int) or token_id < 0) for token_id in token_ids)
            for token_ids in nested_token_ids
        ):
            raise ValueError(
                f"Each list in `nested_token_ids` has to be a list of positive integers, but is {nested_token_ids}."
            )

        self.trie = DisjunctiveTrie(nested_token_ids)
        self.token_ids = nested_token_ids

        self.seqlen = self.trie.max_height
        self.current_seq = []
        self.completed = False

    def advance(self):
        token_list = self.trie.next_tokens(self.current_seq)

        if len(token_list) == 0:
            return None
        else:
            return token_list

    def does_advance(self, token_id: int):
        if not isinstance(token_id, int):
            raise ValueError(f"`token_id` is supposed to be type `int`, but is {token_id} of type {type(token_id)}")

        next_tokens = self.trie.next_tokens(self.current_seq)

        return token_id in next_tokens

    def update(self, token_id: int):
        if not isinstance(token_id, int):
            raise ValueError(f"`token_id` is supposed to be type `int`, but is {token_id} of type {type(token_id)}")

        stepped = False
        completed = False
        reset = False

        if self.does_advance(token_id):
            self.current_seq.append(token_id)
            stepped = True
        else:
            reset = True
            self.reset()

        completed = self.trie.reached_leaf(self.current_seq)
        self.completed = completed

        return stepped, completed, reset

    def reset(self):
        self.completed = False
        self.current_seq = []

    def remaining(self):
        if self.completed:
            # since this can be completed without reaching max height
            return 0
        else:
            return self.seqlen - len(self.current_seq)

    def copy(self, stateful=False):
        new_constraint = DisjunctiveConstraint(self.token_ids)

        if stateful:
            new_constraint.seq_len = self.seqlen
            new_constraint.current_seq = self.current_seq
            new_constraint.completed = self.completed

        return new_constraint
    
    
def get_description_name(idd):
    '''
    This function returns a name of an entity and its description given WikiData id
    
        input:  (str) wikidata id, e.x. 'Q2'
        output: (str) concatenated 'name, description' of a given entity
    '''
    entity = client.get(idd, load=True)
    name = "None"
    description = "None"
    try:
        name = entity.data["labels"]["en"]["value"]
        
    except:
        pass
    return name


def text_to_id(x):
    #splits = x.split(" >> ")
    if len(x) > 1:
        return max(lang_title2wikidataID[tuple(reversed(x))], key=lambda y: int(y[1:]))
    else:
        return 'Empty'
    
def from_text_to_id(x):
    return list(lang_title2wikidataID[("en", x)])[0]
    
    
    
### For top k sample
    
def getScores(ids, scores, pad_token_id):
    """get sequence scores from model.generate output"""
    scores = torch.stack(scores, dim=1)
    log_probs = torch.log_softmax(scores, dim=2)
    # remove start token
    ids = ids[:,1:]
    # gather needed probs
    x = ids.unsqueeze(-1).expand(log_probs.shape)
    needed_logits = torch.gather(log_probs, 2, x)
    final_logits = needed_logits[:, :, 0]
    padded_mask = (ids == pad_token_id)
    final_logits[padded_mask] = 0
    final_scores = final_logits.sum(dim=-1)
    return final_scores.cpu().detach().numpy()

def topkSample(input, model, tokenizer, 
                num_samples=5,
                num_beams=1,
                max_output_length=128):
    tokenized = tokenizer(input, return_tensors="pt")
    tokenized.to(device)
    out = model.generate(**tokenized,
                        do_sample=True,
                        num_return_sequences = num_samples,
                        num_beams = num_beams,
                        eos_token_id = tokenizer.eos_token_id,
                        pad_token_id = tokenizer.pad_token_id,
                        output_scores = True,
                        return_dict_in_generate=True,
                        max_length=max_output_length,)
    out_tokens = out.sequences
    out_str = tokenizer.batch_decode(out_tokens, skip_special_tokens=True)
    out_scores = getScores(out_tokens, out.scores, tokenizer.pad_token_id)
    
    pair_list = [(x[0], x[1]) for x in zip(out_str, out_scores)]
    sorted_pair_list = sorted(pair_list, key=lambda x:x[1], reverse=True)
    return sorted_pair_list

def greedyPredict(input, model, tokenizer):
    input_ids = tokenizer([input], return_tensors="pt").input_ids
    out_tokens = model.generate(input_ids)
    out_str = tokenizer.batch_decode(out_tokens, skip_special_tokens=True)
    return out_str[0]


def check_t5_sq(t5_tok, t5_qa_model):
    
    sq_test_data = np.load("simple_questions_test.npy")
    questions = sq_test_data[:,3]


    answers = []
    for i in tqdm(range(len(questions))):
        input_ids = t5_tok(questions[i], return_tensors="pt").input_ids
        input_ids = input_ids.to(device)
        gen_output = t5_qa_model.generate(input_ids)[0]
        answers.append(gen_output)


    preds_sq = []
    for x in answers:
        preds_sq.append(t5_tok.decode(x, skip_special_tokens=True))


    preds_id_sq = []
    for i in range(len(preds_sq)):
        try:
            x = from_text_to_id(preds_sq[i])
        except:
            x = "None"

        preds_id_sq.append(x)

    right_sq = 0
    for i in tqdm(range(len(preds_id_sq))):
        if preds_id_sq[i] == sq_test_data[i,2]:
            right_sq += 1
        else:
            pass

    acc = right_sq/len(preds_id_sq)
    
    return acc, preds_id_sq

softmax = nn.Softmax()

# Using parameter constraints <- DisjunctiveConstraint

In [20]:
constraints = [DisjunctiveConstraint(nested_token_ids=list(tokenized_data[15:25]), )]

question = ["Which country has won the FIFA World Cup the most times?"]

tokenized_question = tokenizer(question, return_tensors="pt").input_ids.to(device)

outputs = model.generate(
    tokenized_question,
    constraints=constraints,
    num_beams=3,
    num_return_sequences=3,
    max_length=20,
    remove_invalid_values=True,
    early_stopping=True
)

generated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)

start DisjunctiveConstraint
Start DisjunctiveTrie
A little change in Generation function
GOOO
YYYYYYYYYYYYY
start DisjunctiveConstraint
Start DisjunctiveTrie
start DisjunctiveConstraint
Start DisjunctiveTrie
start DisjunctiveConstraint
Start DisjunctiveTrie
start DisjunctiveConstraint
Start DisjunctiveTrie
start DisjunctiveConstraint
Start DisjunctiveTrie
start DisjunctiveConstraint
Start DisjunctiveTrie
start DisjunctiveConstraint
Start DisjunctiveTrie
start DisjunctiveConstraint
Start DisjunctiveTrie
start DisjunctiveConstraint
Start DisjunctiveTrie
start DisjunctiveConstraint
Start DisjunctiveTrie
start DisjunctiveConstraint
Start DisjunctiveTrie
start DisjunctiveConstraint
Start DisjunctiveTrie
start DisjunctiveConstraint
Start DisjunctiveTrie
start DisjunctiveConstraint
Start DisjunctiveTrie
start DisjunctiveConstraint
Start DisjunctiveTrie
start DisjunctiveConstraint
Start DisjunctiveTrie
start DisjunctiveConstraint
Start DisjunctiveTrie
start DisjunctiveConstraint
Start Disjunct

In [21]:
generated_text

['Brazil Union', 'Brazil Yunna Fehér', 'Brazil Yunnan']

In [31]:
tokenized_question

tensor([[ 4073,   684,    65,   751,     8, 23129,  1150,  3802,     8,   167,
           648,    58,     1]], device='cuda:3')

# Comparison with the case without constraint

In [30]:
outputs = model.generate(
    tokenized_question,
    num_beams=5,
    num_return_sequences=5,
    max_length=20,
    early_stopping=True
)
generated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)
generated_text

A little change in Generation function
GOOO
YYYYYYYYYYYYY


['Brazil', 'Germany', 'Australia', 'Mexico', 'Spain']

We can see that here early stoppig works properly

# Let's observe slightly different params

## Without constraint

In [106]:
num_beams = 10
max_sequence_length = 32
question = "Which country has won the FIFA World Cup the most times?"

input_ids = tokenizer([question], return_tensors="pt").input_ids
input_ids = input_ids.to(device)

out = model.generate(input_ids,
                     num_return_sequences = num_beams,
                     num_beams = num_beams,
                     eos_token_id = tokenizer.eos_token_id,
                     pad_token_id = tokenizer.pad_token_id,
                     output_scores = True,
                     return_dict_in_generate=True,
                     early_stopping=True,
                     max_length=max_sequence_length)

prediction = [tokenizer.decode(out.sequences[i], skip_special_tokens=True) for i in range(num_beams)]
probs = softmax(out.sequences_scores)

A little change in Generation function
GOOO
YYYYYYYYYYYYY


In [107]:
prediction, probs

(['Brazil',
  'United States',
  'Brazil (',
  'Germany',
  'Australia',
  'Brazil,',
  'Mexico',
  'Brazil*',
  'Brazil:',
  'Brazil.'],
 tensor([9.8897e-01, 2.6222e-03, 2.3766e-03, 1.5850e-03, 1.2157e-03, 8.3949e-04,
         7.7496e-04, 7.2982e-04, 5.2720e-04, 3.5541e-04], device='cuda:3'))

## With constraint

In [104]:
num_beams = 2
max_sequence_length = 32
question = "Which country has won the FIFA World Cup the most times?"

input_ids = tokenizer([question], return_tensors="pt").input_ids
input_ids = input_ids.to(device)

out = model.generate(input_ids,
                     num_return_sequences = num_beams,
                     num_beams = num_beams,
                     constraints=constraints,
                     eos_token_id = tokenizer.eos_token_id,
                     pad_token_id = tokenizer.pad_token_id,
                     output_scores = True,
                     return_dict_in_generate=True,
                     #max_length=max_sequence_length,
                     early_stopping=True)

prediction = [tokenizer.decode(out.sequences[i], skip_special_tokens=True) for i in range(num_beams)]
probs = softmax(out.sequences_scores)

A little change in Generation function
GOOO
YYYYYYYYYYYYY
start DisjunctiveConstraint
Start DisjunctiveTrie
start DisjunctiveConstraint
Start DisjunctiveTrie
start DisjunctiveConstraint
Start DisjunctiveTrie
start DisjunctiveConstraint
Start DisjunctiveTrie
start DisjunctiveConstraint
Start DisjunctiveTrie
start DisjunctiveConstraint
Start DisjunctiveTrie
start DisjunctiveConstraint
Start DisjunctiveTrie
start DisjunctiveConstraint
Start DisjunctiveTrie
start DisjunctiveConstraint
Start DisjunctiveTrie
start DisjunctiveConstraint
Start DisjunctiveTrie
start DisjunctiveConstraint
Start DisjunctiveTrie
start DisjunctiveConstraint
Start DisjunctiveTrie
start DisjunctiveConstraint
Start DisjunctiveTrie
start DisjunctiveConstraint
Start DisjunctiveTrie
start DisjunctiveConstraint
Start DisjunctiveTrie
start DisjunctiveConstraint
Start DisjunctiveTrie
start DisjunctiveConstraint
Start DisjunctiveTrie
start DisjunctiveConstraint
Start DisjunctiveTrie
start DisjunctiveConstraint
Start Disjunct

In [105]:
prediction, probs

(['Belgiuman Henning Blang', 'Belgium'],
 tensor([0.9980, 0.0020], device='cuda:3'))

In [111]:
trie = DisjunctiveTrie(nested_token_ids=list(tokenized_data[:15]))

Start DisjunctiveTrie


# Now let's try force_word_ids instead constraints

In [10]:
force_word_ids = list(tokenized_data)[:1000]

In [12]:
num_beams = 5
max_sequence_length = 3
question = "Which country has won the FIFA World Cup the most times?"

input_ids = tokenizer([question], return_tensors="pt").input_ids
input_ids = input_ids.to(device)

out = model.generate(input_ids,
                     num_return_sequences = num_beams,
                     num_beams = num_beams,
                     #constraints = constraints,
                     force_words_ids=force_word_ids,
                     eos_token_id = tokenizer.eos_token_id,
                     pad_token_id = tokenizer.pad_token_id,
                     output_scores = True,
                     return_dict_in_generate=True,
                     max_length=max_sequence_length,
                     early_stopping=True)

prediction = [tokenizer.decode(out.sequences[i], skip_special_tokens=True) for i in range(num_beams)]
probs = softmax(out.sequences_scores)

In [13]:
prediction

['Brazil', 'United Arab', 'Mexico City', 'Frances', 'West Ace']

In [15]:
probs

tensor([9.9999e-01, 7.6072e-06, 1.0220e-06, 1.0282e-07, 2.0685e-13],
       device='cuda:3')

In [12]:
num_beams = 5
max_sequence_length = 3
question = "Which country has won the FIFA World Cup the most times?"

input_ids = tokenizer([question], return_tensors="pt").input_ids
input_ids = input_ids.to(device)

out = model.generate(input_ids,
                     num_return_sequences = num_beams,
                     num_beams = num_beams,
                     #constraints = constraints,
                     force_words_ids=force_word_ids,
                     eos_token_id = tokenizer.eos_token_id,
                     pad_token_id = tokenizer.pad_token_id,
                     output_scores = True,
                     return_dict_in_generate=True,
                     max_length=max_sequence_length,
                     early_stopping=True)

prediction = [tokenizer.decode(out.sequences[i], skip_special_tokens=True) for i in range(num_beams)]
probs = softmax(out.sequences_scores)

# not successsful attempt to enforce stopping

In [16]:
from transformers import  StoppingCriteriaList

In [14]:
tokenizer.decode(tokenizer.encode("Hai"))

'Hai</s>'

In [48]:
num_beams = 5
max_sequence_length = 4
#StoppingCriteriaList(['</s>'])

question = "Which country has won the FIFA World Cup the most times?"

input_ids = tokenizer([question], return_tensors="pt").input_ids
input_ids = input_ids.to(device)

out = model.generate(input_ids,
                     num_return_sequences = num_beams,
                     num_beams = num_beams,
                     #constraints = constraints,
                     force_words_ids=force_word_ids,
                     eos_token_id = tokenizer.eos_token_id,
                     pad_token_id = tokenizer.pad_token_id,
                     output_scores = True,
                     return_dict_in_generate=True,
                     max_length=max_sequence_length,
                     early_stopping=True,
                     length_penalty = -0.5,
                     stopping_criteria=)

prediction = [tokenizer.decode(out.sequences[i], skip_special_tokens=True) for i in range(num_beams)]
probs = softmax(out.sequences_scores)

print(prediction)
print(probs)

['Brazil Union', 'Brazil Con', 'Brazil U', 'Brazil Republic', 'Brazil L']
tensor([8.8343e-01, 1.1656e-01, 8.9519e-06, 4.0635e-07, 1.7818e-09],
       device='cuda:3')


# probabilities below are slightly differnet since I tried to add length_penalty (less value -> more penalty for length)

In [26]:
"Chile" in [tokenizer.decode(i, skip_special_tokens=True) for i in force_word_ids]

True

In [13]:
prediction, probs

(['Brazil', 'United Arab', 'Mexico City', 'Frances', 'West Ace'],
 tensor([9.9999e-01, 7.6072e-06, 1.0220e-06, 1.0282e-07, 2.0685e-13],
        device='cuda:3'))

# Try to make constraint using prefix_allowed_tokens_fn

In [ ]:
!pip install marisa-trie

In [53]:
import marisa_trie

In [115]:
d_trie = DisjunctiveTrie(list(tokenized_data[:1000])).trie

Start DisjunctiveTrie


In [119]:
d_trie[15575]

{1: {}}

In [135]:
def trie_func(batch_id, sent):
    print("batch_id = ", batch_id, "\n")
    print("sent = ", sent.tolist(), "\n")
    return d_trie.get(sent)

In [138]:
num_beams = 10
max_sequence_length = 4
#StoppingCriteriaList(['</s>'])

question = "What instrument John Lennon played?"

input_ids = tokenizer([question], return_tensors="pt").input_ids
input_ids = input_ids.to(device)

out = model.generate(input_ids,
                     num_return_sequences = num_beams,
                     num_beams = num_beams,
                     #constraints = constraints,
                     force_words_ids=force_word_ids,
                     eos_token_id = tokenizer.eos_token_id,
                     pad_token_id = tokenizer.pad_token_id,
                     output_scores = True,
                     return_dict_in_generate=True,
                     max_length=max_sequence_length,
                     early_stopping=True,
                     #length_penalty = -1.0,
                     prefix_allowed_tokens_fn = trie_func)

prediction = [tokenizer.decode(out.sequences[i], skip_special_tokens=True) for i in range(num_beams)]
probs = softmax(out.sequences_scores)

print(prediction)
print(probs)

batch_id =  0 

sent =  [0] 

batch_id =  0 

sent =  [0] 

batch_id =  0 

sent =  [0] 

batch_id =  0 

sent =  [0] 

batch_id =  0 

sent =  [0] 

batch_id =  0 

sent =  [0] 

batch_id =  0 

sent =  [0] 

batch_id =  0 

sent =  [0] 

batch_id =  0 

sent =  [0] 

batch_id =  0 

sent =  [0] 



KeyboardInterrupt: 

In [124]:
res = [tokenizer.decode(i, skip_special_tokens=True) for i in list(tokenized_data[:1000])]

In [131]:
tokenizer.decode([0, 5507], skip_special_tokens=True)

'guitar'

In [11]:
from GENRE.genre.trie import Trie, MarisaTrie

In [11]:
%pwd

'/home/petrakov/Experiments_with_UE'

In [14]:
! wget http://dl.fbaipublicfiles.com/GENRE/titles_lang_all105_trie_with_redirect.pkl

--2022-11-24 09:10:00--  http://dl.fbaipublicfiles.com/GENRE/titles_lang_all105_trie_with_redirect.pkl
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.75.142, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2338474677 (2.2G) [application/octet-stream]
Saving to: ‘titles_lang_all105_trie_with_redirect.pkl’

titles_lang_all105_ 100%[===================>]   2.18G  23.1MB/s    in 99s     

2022-11-24 09:11:40 (22.6 MB/s) - ‘titles_lang_all105_trie_with_redirect.pkl’ saved [2338474677/2338474677]



# mgenre|

In [9]:
import marisa_trie

In [12]:
with open("titles_lang_all105_marisa_trie_with_redirect.pkl", "rb") as f:
    trie_2 = pickle.load(f)

In [14]:
trie_2_dict = dict(trie_2.trie)

In [19]:
trie_2_dict

{'\x02\U0001189e\U0003e8e4\U0003e8e9〨)Þ§≭𗭁Þ§턎簋\\\U0003e8e6\u0ee1ၑ\U0003e8e8ⶎ⁝Έ\x02': 88844519,
 '\x02\U0001189e\U0003e8e4\U0003e8e9〨)Þ§≭𗭁Þ§턎簋\\\U0003e8e6\u0ee1\U0001b6c5\U0003e8e8ⶎ⁝Έ\x02': 88844520,
 '\x02\U0001189e\U0003e8e4\U0003e8e9〨)Þ§≭𗭁Þ§턎簋\\\U0003e8e6\u0bbb\U0003ea9b˱\U0003e8e8ⶎ⁝Έ\x02': 88842918,
 '\x02\U0001189e\U0003e8e4\U0003e8e9〨)Þ§≭𗭁Þ§턎簋\\\U0003e8e6๒㒯\U0003e8e8ⶎ⁝Έ\x02': 88842919,
 '\x02\U0001189e\U0003e8e4\U0003e8e9〨)Þ§≭𗭁Þ§턎簋\\\U0003e8e6𗤨宜˱\U0003e8e8ⶎ⁝Έ\x02': 88842920,
 '\x02\U0001189e\U0003e8e4\U0003e8e9〨)Þ§≭𗭁Þ§턎簋\\\U0003e8e6\U0003ea9bĞ\U0003e8fb\U0003e8e8ⶎ⁝Έ\x02': 88842921,
 '\x02\U0001189e\U0003e8e4\U0003e8e9〨)Þ§≭𗭁Þ§턎簋\\\U0003e8e6\x8c\x82㱨\U0003e8e8ⶎ⁝Έ\x02': 88840357,
 '\x02\U0001189e\U0003e8e4\U0003e8e9〨)Þ§≭𗭁Þ§턎簋\\\U0003e8e6͟\u0de4ÿ\U0003e8e8ⶎ⁝Έ\x02': 88840358,
 '\x02\U0001189e\U0003e8e4\U0003e8e9〨)Þ§≭𗭁Þ§턎簋\\\U0003e8e6ύⲚÿ\U0003e8e8ⶎ⁝Έ\x02': 88840359,
 '\x02\U0001189e\U0003e8e4\U0003e8e9〨)Þ§≭𗭁Þ§턎簋\\\U0003e8e6\u058c\U0003e8d3\U0003ed3f\U0003e8e8ⶎ⁝Έ\x02': 88840360,
 '\x02\U

In [15]:
with open("titles_lang_all105_trie_with_redirect.pkl", "rb") as f:
    trie_1 = Trie.load_from_dict(pickle.load(f))

In [34]:
len(trie_1)

76658274

In [23]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# OPTIONAL: load the prefix tree (trie), you need to additionally download
# https://huggingface.co/facebook/genre-kilt/blob/main/trie.py and 
# https://huggingface.co/facebook/genre-kilt/blob/main/kilt_titles_trie_dict.pkl
# import pickle
# from trie import Trie
# with open("kilt_titles_trie_dict.pkl", "rb") as f:
#     trie = Trie.load_from_dict(pickle.load(f))

tokenizer = AutoTokenizer.from_pretrained("facebook/genre-kilt")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/genre-kilt").eval()

sentences = ["Einstein was a German physicist."]



In [24]:
outputs = model.generate(
    **tokenizer(sentences, return_tensors="pt"),
    num_beams=5,
    num_return_sequences=5,
    output_scores = True,
    return_dict_in_generate=True,
    # OPTIONAL: use constrained beam search
    prefix_allowed_tokens_fn=lambda batch_id, sent: trie_1.get(sent.tolist()),
)



IndexError: index 144570 is out of bounds for dimension 0 with size 50264

In [ ]:
softmax(outputs.sequences_scores)

In [ ]:
tokenizer.batch_decode(outputs.sequences, skip_special_tokens=True)

# Reproduction of HF cite code

In [133]:
constraints = [DisjunctiveConstraint(nested_token_ids=list(tokenized_data[:30]))]
#constraints = [PhrasalConstraint(token_ids = list(tokenized_data[:15][0]))]
constraints

start DisjunctiveConstraint
Start DisjunctiveTrie


In [71]:
from transformers import PhrasalConstraint

In [42]:
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
question = "Which country has won the FIFA World Cup the most times?"


#encoder_input_str = "translate English to German: How old are you?"
encoder_input_ids = tokenizer([question],return_tensors="pt").input_ids
encoder_input_ids = encoder_input_ids.to(device)


# lets run beam search using 3 beams
num_beams = 10
# define decoder start token ids
input_ids = torch.ones((num_beams, 1), device=model.device, dtype=torch.long)
input_ids = input_ids * model.config.decoder_start_token_id


# add encoder_outputs to model keyword arguments
model_kwargs = {
    "encoder_outputs": model.get_encoder()(
        encoder_input_ids.repeat_interleave(num_beams, dim=0), return_dict=True
    )
}

#input_ids = tokenizer([question], return_tensors="pt").input_ids


beam_scorer = ConstrainedBeamSearchScorer(
    batch_size=1, num_beams=num_beams, device=model.device, constraints=constraints
)

# instantiate logits processors
logits_processor = LogitsProcessorList(
    [
        MinLengthLogitsProcessor(5, eos_token_id=model.config.eos_token_id),
    ]
)

outputs = model.constrained_beam_search(
    input_ids, beam_scorer, constraints=constraints, logits_processor=logits_processor, **model_kwargs
)

final = tokenizer.batch_decode(outputs, skip_special_tokens=True)

Start DisjunctiveTrie
Start DisjunctiveTrie
Start DisjunctiveTrie
Start DisjunctiveTrie
Start DisjunctiveTrie
Start DisjunctiveTrie
Start DisjunctiveTrie
Start DisjunctiveTrie
Start DisjunctiveTrie
Start DisjunctiveTrie
Start DisjunctiveTrie
Start DisjunctiveTrie
Start DisjunctiveTrie
Start DisjunctiveTrie
Start DisjunctiveTrie
Start DisjunctiveTrie
Start DisjunctiveTrie
Start DisjunctiveTrie
Start DisjunctiveTrie
Start DisjunctiveTrie
Start DisjunctiveTrie
Start DisjunctiveTrie
Start DisjunctiveTrie
Start DisjunctiveTrie
Start DisjunctiveTrie
Start DisjunctiveTrie
Start DisjunctiveTrie
Start DisjunctiveTrie
Start DisjunctiveTrie
Start DisjunctiveTrie
Start DisjunctiveTrie
Start DisjunctiveTrie
Start DisjunctiveTrie
Start DisjunctiveTrie
Start DisjunctiveTrie
Start DisjunctiveTrie
Start DisjunctiveTrie
Start DisjunctiveTrie
Start DisjunctiveTrie
Start DisjunctiveTrie
Start DisjunctiveTrie
Start DisjunctiveTrie
Start DisjunctiveTrie
Start DisjunctiveTrie
Start DisjunctiveTrie
Start Disj

In [95]:
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
question = "Which country has won the FIFA World Cup the most times?"
batch_size = 2
num_beams = 3


#encoder_input_str = "translate English to German: How old are you?"
encoder_input_ids = tokenizer([question],return_tensors="pt").input_ids
encoder_input_ids = encoder_input_ids.to(device)


# lets run beam search using 3 beams

# define decoder start token ids
input_ids = torch.ones((num_beams * batch_size, 1), device=model.device, dtype=torch.long)
input_ids = input_ids * model.config.decoder_start_token_id


#add encoder_outputs to model keyword arguments
model_kwargs = {
    "encoder_outputs": model.get_encoder()(
        encoder_input_ids.repeat_interleave(num_beams, dim=0), return_dict=True
    )
}




beam_scorer = ConstrainedBeamSearchScorer(
    batch_size=batch_size, num_beams=num_beams, device=model.device, constraints=constraints
)


outputs = model.constrained_beam_search(input_ids, beam_scorer, constraints=constraints, **model_kwargs)

final = tokenizer.batch_decode(outputs, skip_special_tokens=True)

RuntimeError: shape '[6, -1, 6, 64]' is invalid for input of size 14976

In [91]:
final

['Brazil Union']

In [67]:
with open("lang_title2wikidataID-normalized_with_redirect.pkl", "rb") as f:
    lang_title2wikidataID = pickle.load(f)

In [79]:
def text_to_id(x):
    #splits = x.split(" >> ")
    if len(x) > 1:
        return max(lang_title2wikidataID[tuple(reversed(x))], key=lambda y: int(y[1:]))
    else:
        return 'Empty'
    
def from_text_to_id(x):
    try:
        return list(lang_title2wikidataID[("en", x)])[0]
    except:
        return "None"
    